# Ollama + LlamaIndex Testing Notebook

This notebook tests:
- Ollama Python library
- Downloading llama3.2:1b model (1.5B parameters)
- Reading prompts from file
- Context management
- LlamaIndex integration

## 1. Install Required Libraries

In [2]:
# Install required packages
!pip install ollama llama-index llama-index-llms-ollama llama-index-embeddings-ollama

Defaulting to user installation because normal site-packages is not writeable


## 2. Import Libraries

In [3]:
import ollama
import os
from pathlib import Path
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.memory import ChatMemoryBuffer

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## 3. Create Folder Structure and Sample Files

In [4]:
# Create dedicated model folder
model_folder = Path("./ollama_test")
model_folder.mkdir(exist_ok=True)

# Create prompt.txt file
prompt_file = model_folder / "prompt.txt"
with open(prompt_file, "w") as f:
    f.write("""You are a helpful AI assistant. Answer questions concisely and accurately.
Context: You are helping a developer test an agentic chatbot system with model switching capabilities.
Task: Answer the user's questions while being aware of the context provided.""")

# Create context.txt file
context_file = model_folder / "context.txt"
with open(context_file, "w") as f:
    f.write("""Project Context:
- Building an agentic offline chatbot
- Using FastAPI backend with React frontend
- PostgreSQL for storing conversations
- Model switching capability between different LLMs
- Using Ollama for running local models""")

print(f"✓ Created folder: {model_folder}")
print(f"✓ Created: prompt.txt")
print(f"✓ Created: context.txt")

✓ Created folder: ollama_test
✓ Created: prompt.txt
✓ Created: context.txt


## 4. Download Llama 1.5B Model (llama3.2:1b)

In [5]:
# Check if Ollama is running
try:
    ollama.list()
    print("✓ Ollama is running!")
except Exception as e:
    print("❌ Ollama is not running. Please start Ollama first.")
    print("Run: ollama serve")
    raise e

✓ Ollama is running!


In [7]:
# Download llama3.2:1b model (1.5B parameters)
model_name = "llama3.2:1b"

print(f"Downloading {model_name}... This may take a few minutes.")
try:
    # Pull the model
    ollama.pull(model_name)
    print(f"✓ Model {model_name} downloaded successfully!")
except Exception as e:
    print(f"Error downloading model: {e}")
    print("\nAlternative models you can try:")
    print("- llama3.2:1b (1.5B params)")
    print("- llama3.2:3b (3B params)")
    print("- phi3:mini (3.8B params)")

✓ Model llama3.2:1b downloaded successfully!


In [9]:
# List available models (FIXED)
models = ollama.list()
print("\n📋 Available models:")

# Handle different response structures
if hasattr(models, 'models'):
    model_list = models.models if hasattr(models.models, '__iter__') else models['models']
else:
    model_list = models.get('models', [])

for model in model_list:
    # Try different possible attribute names
    model_name_attr = getattr(model, 'model', None) or getattr(model, 'name', None)
    model_size = getattr(model, 'size', 0)
    
    if model_name_attr:
        size_gb = model_size / (1024**3) if model_size > 0 else 0
        print(f"  - {model_name_attr} (Size: {size_gb:.2f} GB)")
    else:
        # Fallback: just print the model object
        print(f"  - {model}")


📋 Available models:
  - llama3.2:1b (Size: 1.23 GB)


## 5. Test Basic Ollama Usage with Prompt File

In [10]:
# Read prompt from file
with open(prompt_file, "r") as f:
    system_prompt = f.read()

# Read context from file
with open(context_file, "r") as f:
    context = f.read()

print("System Prompt:")
print(system_prompt)
print("\nContext:")
print(context)

System Prompt:
You are a helpful AI assistant. Answer questions concisely and accurately.
Context: You are helping a developer test an agentic chatbot system with model switching capabilities.
Task: Answer the user's questions while being aware of the context provided.

Context:
Project Context:
- Building an agentic offline chatbot
- Using FastAPI backend with React frontend
- PostgreSQL for storing conversations
- Model switching capability between different LLMs
- Using Ollama for running local models


In [11]:
# Test basic chat with context
messages = [
    {
        'role': 'system',
        'content': system_prompt
    },
    {
        'role': 'user',
        'content': f"Context: {context}\n\nQuestion: What kind of chatbot are we building?"
    }
]

print("\n🤖 Testing Basic Ollama Chat...\n")
response = ollama.chat(model=model_name, messages=messages)
print(f"Response: {response['message']['content']}")


🤖 Testing Basic Ollama Chat...

Response: Based on the context provided, it appears that you're developing an agentic (self-aware) offline chatbot. This type of chatbot is designed to understand and respond to natural language inputs, often in a conversational manner.

The fact that you're using FastAPI as your backend API and React as the frontend suggests a modern and user-friendly interface. The PostgreSQL database for storing conversations implies a robust and scalable data management system.

Model switching capability between different LLMs (Large Language Models) is also an interesting aspect, which further supports the agentic nature of your chatbot. This ability to switch models in response to different conversation topics or user queries will allow you to adapt to various situations and improve the overall experience for users.

Lastly, using Ollama for running local models on a Raspberry Pi (or another device) adds an extra layer of customizability, allowing you to fine-tun

## 6. Test Streaming Response

In [12]:
# Test streaming
print("\n🤖 Testing Streaming Response...\n")
messages.append({
    'role': 'user',
    'content': 'Explain what FastAPI is in one sentence.'
})

stream = ollama.chat(
    model=model_name,
    messages=messages,
    stream=True
)

full_response = ""
for chunk in stream:
    content = chunk['message']['content']
    print(content, end='', flush=True)
    full_response += content

print("\n")


🤖 Testing Streaming Response...

FastAPI is a modern, fast (high-performance), web framework written in Python that allows developers to build APIs quickly and efficiently.

As for your question about the type of chatbot, you're thinking of creating an agentic offline chatbot, which means using a conversational AI system that can engage with users over an extended period without requiring internet connectivity, such as a voice-controlled or smart home device.



## 7. LlamaIndex Integration

In [13]:
# Configure LlamaIndex with Ollama
llm = Ollama(model=model_name, request_timeout=120.0)
embed_model = OllamaEmbedding(model_name="nomic-embed-text")  # Smaller embedding model

# Set as default
Settings.llm = llm
Settings.embed_model = embed_model

print("✓ LlamaIndex configured with Ollama")

✓ LlamaIndex configured with Ollama


In [14]:
# Download embedding model if needed
try:
    ollama.pull("nomic-embed-text")
    print("✓ Embedding model ready")
except Exception as e:
    print(f"Note: {e}")

✓ Embedding model ready


In [15]:
# Create a simple chat engine with memory
from llama_index.core.chat_engine import SimpleChatEngine

chat_engine = SimpleChatEngine.from_defaults(
    llm=llm,
    system_prompt=system_prompt
)

print("✓ Chat engine created with memory")

✓ Chat engine created with memory


In [16]:
# Test chat with context memory
print("\n🤖 Testing LlamaIndex Chat Engine...\n")

# First message
response1 = chat_engine.chat(f"Context: {context}\n\nWhat database are we using?")
print(f"Q1: What database are we using?")
print(f"A1: {response1.response}\n")

# Second message (should remember context)
response2 = chat_engine.chat("What frontend framework did I mention?")
print(f"Q2: What frontend framework did I mention?")
print(f"A2: {response2.response}\n")


🤖 Testing LlamaIndex Chat Engine...

Q1: What database are we using?
A1: We're using PostgreSQL as the relational database to store our conversations.

Q2: What frontend framework did I mention?
A2: You mentioned React as the frontend framework, specifically for building a chatbot with FastAPI backend and PostgreSQL database.



## 8. Document Indexing with LlamaIndex (RAG)

In [17]:
# Create sample documents for RAG
docs_folder = model_folder / "docs"
docs_folder.mkdir(exist_ok=True)

# Create sample documents
with open(docs_folder / "fastapi_info.txt", "w") as f:
    f.write("""FastAPI is a modern, fast web framework for building APIs with Python.
It supports async/await, automatic API documentation, and type hints.
FastAPI is built on Starlette and Pydantic.""")

with open(docs_folder / "ollama_info.txt", "w") as f:
    f.write("""Ollama is a tool for running large language models locally.
It supports models like Llama, Mistral, and many others.
Ollama provides a simple API and command-line interface.""")

print("✓ Sample documents created")

✓ Sample documents created


In [18]:
# Index documents
print("\n📚 Indexing documents...")
documents = SimpleDirectoryReader(str(docs_folder)).load_data()
index = VectorStoreIndex.from_documents(documents)
print(f"✓ Indexed {len(documents)} documents")


📚 Indexing documents...


2026-01-22 12:49:38,442 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


✓ Indexed 2 documents


In [19]:
# Query the index (RAG)
query_engine = index.as_query_engine()

print("\n🔍 Testing RAG Query...\n")
response = query_engine.query("What is FastAPI built on?")
print(f"Question: What is FastAPI built on?")
print(f"Answer: {response.response}")


🔍 Testing RAG Query...



2026-01-22 12:49:54,019 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-22 12:50:12,187 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Question: What is FastAPI built on?
Answer: FastAPI is built using the following frameworks and libraries:

1. Starlette
2. Pydantic


## 9. Model Switching Test

In [20]:
# Simulate model switching with context preservation
class ModelSwitcher:
    def __init__(self):
        self.conversation_history = []
        self.current_model = model_name
    
    def chat(self, user_message, model=None):
        if model and model != self.current_model:
            print(f"\n🔄 Switching from {self.current_model} to {model}")
            self.current_model = model
        
        # Add user message to history
        self.conversation_history.append({
            'role': 'user',
            'content': user_message
        })
        
        # Get response
        messages = [
            {'role': 'system', 'content': system_prompt}
        ] + self.conversation_history
        
        response = ollama.chat(
            model=self.current_model,
            messages=messages
        )
        
        # Add assistant response to history
        assistant_message = response['message']['content']
        self.conversation_history.append({
            'role': 'assistant',
            'content': assistant_message
        })
        
        return assistant_message
    
    def get_context(self):
        return self.conversation_history

# Test model switcher
switcher = ModelSwitcher()
print("✓ Model switcher initialized")

✓ Model switcher initialized


In [21]:
# Test conversation with model switching
print("\n💬 Testing Model Switching with Context...\n")

response1 = switcher.chat("My name is John")
print(f"Model: {switcher.current_model}")
print(f"User: My name is John")
print(f"Assistant: {response1}\n")

response2 = switcher.chat("What's my name?")  # Should remember
print(f"Model: {switcher.current_model}")
print(f"User: What's my name?")
print(f"Assistant: {response2}\n")

# Show context is preserved
print("\n📝 Conversation History:")
for msg in switcher.get_context():
    print(f"  {msg['role']}: {msg['content'][:50]}...")


💬 Testing Model Switching with Context...



2026-01-22 12:50:23,707 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Model: llama3.2:1b
User: My name is John
Assistant: Hello John, I'm here to help you with your chatbot system. How can I assist you today? Are you experiencing any issues or need help with anything specific regarding your agentic chatbot system?



2026-01-22 12:50:29,202 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Model: llama3.2:1b
User: What's my name?
Assistant: Your name is John. How can I help you further, John? Is there a question or concern about your chatbot system that you'd like to discuss?


📝 Conversation History:
  user: My name is John...
  assistant: Hello John, I'm here to help you with your chatbot...
  user: What's my name?...
  assistant: Your name is John. How can I help you further, Joh...


## 10. Summary & Next Steps

In [22]:
print("""
✅ Testing Complete!

What we tested:
1. ✓ Ollama Python library
2. ✓ Downloaded llama3.2:1b model
3. ✓ Read prompts from file
4. ✓ Context management
5. ✓ Streaming responses
6. ✓ LlamaIndex integration
7. ✓ RAG with document indexing
8. ✓ Model switching with context preservation

Next Steps for FastAPI Integration:
1. Create FastAPI endpoints for chat
2. Add WebSocket for streaming
3. Integrate PostgreSQL for persistence
4. Add conversation management
5. Implement agent tools/functions
6. Build React frontend

Files created:
- ./ollama_test/prompt.txt
- ./ollama_test/context.txt
- ./ollama_test/docs/fastapi_info.txt
- ./ollama_test/docs/ollama_info.txt
""")


✅ Testing Complete!

What we tested:
1. ✓ Ollama Python library
2. ✓ Downloaded llama3.2:1b model
3. ✓ Read prompts from file
4. ✓ Context management
5. ✓ Streaming responses
6. ✓ LlamaIndex integration
7. ✓ RAG with document indexing
8. ✓ Model switching with context preservation

Next Steps for FastAPI Integration:
1. Create FastAPI endpoints for chat
2. Add WebSocket for streaming
3. Integrate PostgreSQL for persistence
4. Add conversation management
5. Implement agent tools/functions
6. Build React frontend

Files created:
- ./ollama_test/prompt.txt
- ./ollama_test/context.txt
- ./ollama_test/docs/fastapi_info.txt
- ./ollama_test/docs/ollama_info.txt

